In [198]:
import pandas as pd 
import datetime
import os

In [199]:
cities = pd.read_csv('data/cities.csv')
providers = pd.read_csv('data/providers.csv')
stations = pd.read_csv('data/stations.csv')
tickets = pd.read_csv('data/ticket_data.csv')

# Clean Data

## Drop NaN rows

In [200]:
cities = cities.dropna(how="all")
providers = providers.dropna(how="all")
stations = stations.dropna(how="all")
tickets = tickets.dropna(how="all")

# Extraire des infos intéressantes

## Prix min, moyen et max

In [201]:
min = tickets.describe().price_in_cents["min"]
moy = tickets.describe().price_in_cents["mean"]
max = tickets.describe().price_in_cents["max"]

print(min , moy, max, sep=", ")

300.0, 4382.711061374178, 38550.0


## Durée min/max/moyenne par trajet

### calcule de duree

In [202]:
arrive = [datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S+%f') for date in tickets['arrival_ts']]
depart = [datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S+%f') for date in tickets['departure_ts']]

In [203]:
arrive = pd.DataFrame(arrive)
depart = pd.DataFrame(depart)

In [204]:
tickets['duree'] = 1;

In [205]:
tickets['duree'] = arrive - depart

### Durée max par trajet

In [207]:
tickets_max = tickets.groupby(['o_station','d_station']).max()
#tickets_max.head()

id  company            departure_ts  \
o_station d_station                                             
3.0       39.0       6848666       13  2017-10-26 22:10:00+00   
          61.0       6849450       13  2017-10-29 00:50:00+00   
          123.0      6862926       13  2017-10-19 22:10:00+00   
          132.0      6854174       13  2017-10-20 12:20:00+00   
          189.0      6854672       13  2017-10-30 00:50:00+00   

                                 arrival_ts  price_in_cents  \
o_station d_station                                           
3.0       39.0       2017-10-27 04:25:00+00            3480   
          61.0       2017-10-29 20:05:00+00            4780   
          123.0      2017-10-20 20:40:00+00            3600   
          132.0      2017-10-20 21:35:00+00            2280   
          189.0      2017-10-30 15:25:00+00            3680   

                                      search_ts middle_stations  \
o_station d_station                                               
3.0       39.0       2017-10-14 22:45:48.084+00       {396,396}   
          61.0       2017-10-15 11:12:21.294+00       {586,586}   
          123.0      2017-10-18 12:05:00.173+00       {554,554}   
          132.0      2017-10-16 13:50:39.791+00       {396,396}   
          189.0      2017-10-16 15:48:53.814+00       {586,586}   

                    other_companies  o_city  d_city    duree  
o_station d_station                                           
3.0       39.0                 {13}     628      37 09:35:00  
          61.0                 {13}     628     173 19:15:00  
          123.0                {13}     628     144 22:30:00  
          132.0                 {9}     628     158 09:15:00  
          189.0                {13}     628    5118 14:35:00

### Durée min par trajet

In [208]:
tickets_min = tickets.groupby(['o_station','d_station']).min()
tickets_min.head()

id  company            departure_ts  \
o_station d_station                                             
3.0       39.0       6840169        9  2017-10-19 22:10:00+00   
          61.0       6849450       13  2017-10-29 00:50:00+00   
          123.0      6862926       13  2017-10-19 22:10:00+00   
          132.0      6854165       13  2017-10-20 04:15:00+00   
          189.0      6854587       13  2017-10-30 00:50:00+00   

                                 arrival_ts  price_in_cents  \
o_station d_station                                           
3.0       39.0       2017-10-20 04:25:00+00            1200   
          61.0       2017-10-29 20:05:00+00            4780   
          123.0      2017-10-20 20:40:00+00            3600   
          132.0      2017-10-20 09:15:00+00            1890   
          189.0      2017-10-30 15:25:00+00            3680   

                                      search_ts middle_stations  \
o_station d_station                                               
3.0       39.0       2017-10-12 13:24:39.174+00       {396,396}   
          61.0       2017-10-15 11:12:21.294+00       {586,586}   
          123.0      2017-10-18 12:05:00.173+00       {554,554}   
          132.0       2017-10-16 13:50:39.79+00   {10894,10894}   
          189.0      2017-10-16 15:36:16.535+00       {586,586}   

                    other_companies  o_city  d_city    duree  
o_station d_station                                           
3.0       39.0                 {13}     628      37 06:15:00  
          61.0                 {13}     628     173 19:15:00  
          123.0                {13}     628     144 22:30:00  
          132.0                {13}     628     158 05:00:00  
          189.0                {13}     628    5118 14:35:00

### Durée moy par trajet

In [213]:
tickets_moy = tickets.groupby(['o_station','d_station','duree']).mean()
tickets_moy.head()

id  company  price_in_cents  o_city  \
o_station d_station duree                                                  
3.0       39.0      06:15:00  6844423.0     13.0          3280.0   628.0   
                    06:35:00  6840175.0      9.0          1600.0   628.0   
                    06:45:00  6840179.0     13.0          1590.0   628.0   
                    07:20:00  6840176.0     13.0          1200.0   628.0   
                    09:35:00  6840169.0     13.0          2480.0   628.0   

                              d_city  
o_station d_station duree             
3.0       39.0      06:15:00    37.0  
                    06:35:00    37.0  
                    06:45:00    37.0  
                    07:20:00    37.0  
                    09:35:00    37.0

### Enregistré

In [214]:
# if os.path.isfile('data/modified/tickets_mean.csv') == False :
#      tickets_mean.to_csv('data/modified/tickets.csv')

## Différence de prix moyen et durée selon le train, le bus et le covoit
## selon la distance du trajet (0-200km, 201-800km, 800-2000km, 2000+km)

### Standardisation des types


In [216]:
#providers.loc[providers['transport_type'].str.contains('car'), ['transport_type']] = 'carpooling'

### inserer les type dans la table tickets

In [217]:
tickets['type'] = 1

In [218]:
## return transport_type apartir du company id
# providers[providers['id'] == row['company']]['transport_type'].unique()

types = pd.DataFrame()
#take a lot of time
for index, row in tickets.iterrows():
    value = providers[providers['id'] == row['company']]['transport_type'].unique()[0]
    types = types.append({'type': [value]}, ignore_index=True)
    

KeyboardInterrupt: 

In [219]:
# types.head()

In [220]:
tickets['type'] = types

In [221]:
## transform from list to str
tickets['type'] = tickets['type'].apply(', '.join)

TypeError: can only join an iterable

In [150]:
# tickets.head()

#### prix et duree moyen selon le train, le bus et le covoit

In [222]:
typs_grouped = tickets[['type', 'duree', 'price_in_cents']].groupby(['type']).mean(numeric_only=False)
typs_grouped

TypeError: unhashable type: 'list'

## j'ai pas arrivé a extraire les distances.